Python Decorators (Advanced)
------------------------------------------

The decorator pattern allows behavior to be added to an individual object, dynamically, without affecting the behavior of other objects from the same class.

The decorator pattern is often useful for adhering to the Single Responsibility Principle, as it allows functionality to be divided between classes with unique areas of concern. 

The decorator pattern also helps the developer to adhere to the DRY principle; Don't Repeat Yourself: Eliminate duplicated code.


NOTE: Decorators are a meta-programming concept as their job is to "generate" code.


In [1]:

class MySillyClass:
    """
    A silly class; perfect for a tutorial
    """

    def foo(self):
        """
        Beautfil is better than ugly.
        """
        print("MySillyClass.foo() called")
        return "foo"

    def bar(self):
        """
        Explicit is better than implicit.
        """
        print("MySillyClass.bar() called")
        return "bar"

    def baz(self):
        """
        Simple is better than complex.
        """
        print("MySillyClass.baz() called")
        return "baz"
    

silly = MySillyClass()
print(f"silly.foo = {silly.foo()}\n")
print(f"silly.bar = {silly.bar()}\n")
print(f"silly.baz = {silly.baz()}\n")


MySillyClass.foo() called
silly.foo = foo

MySillyClass.bar() called
silly.bar = bar

MySillyClass.baz() called
silly.baz = baz




Lets have a quick look at the documentation for our silly class, later, you'll see why this is important.


In [2]:

help(MySillyClass)


Help on class MySillyClass in module __main__:

class MySillyClass(builtins.object)
 |  A silly class; perfect for a tutorial
 |  
 |  Methods defined here:
 |  
 |  bar(self)
 |      Explicit is better than implicit.
 |  
 |  baz(self)
 |      Simple is better than complex.
 |  
 |  foo(self)
 |      Beautfil is better than ugly.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)




The silly class contains duplicate code - each of its methods starts with the same line - printing the name of the method.

Using a method decorator is a neat solution we can adopt to remove this duplicated code:


In [3]:

def my_decorator(func):
    """
    Decorator to add information about which function was called
    """
    def wrapper(*args, **kwargs):
        print(f"{func.__qualname__}")
        return func(*args, **kwargs)
    return wrapper


class MySillyClass:
    """
    A silly class; perfect for a tutorial
    """

    @my_decorator
    def foo(self):
        """
        Beautfil is better than ugly.
        """
        return "foo"

    @my_decorator
    def bar(self):
        """
        Explicit is better than implicit.
        """
        return "bar"

    @my_decorator
    def baz(self):
        """
        Simple is better than complex.
        """
        return "baz"
    

silly = MySillyClass()
print(f"silly.foo = {silly.foo()}\n")
print(f"silly.bar = {silly.bar()}\n")
print(f"silly.baz = {silly.baz()}\n")


MySillyClass.foo
silly.foo = foo

MySillyClass.bar
silly.bar = bar

MySillyClass.baz
silly.baz = baz




Lets have another quick look at the documentation for our silly class:


In [4]:

help(MySillyClass)


Help on class MySillyClass in module __main__:

class MySillyClass(builtins.object)
 |  A silly class; perfect for a tutorial
 |  
 |  Methods defined here:
 |  
 |  bar = wrapper(*args, **kwargs)
 |  
 |  baz = wrapper(*args, **kwargs)
 |  
 |  foo = wrapper(*args, **kwargs)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)




What's happened!? ...That's probably not what you expected!

The reason is that the decorated method has lost it's original metadata. Metadata for a Python object is the built-in stuff such as:
__name__, __doc__, __module__, __repr__, __str__ etc...

What we are seeing here is the metadata for the wrapper function - which has been defined inside our decorator. So how do you get the correct metadata back?

Thankfully, this is Python, and the answer is simple, you rely on the standard functools module which has a useful function called wraps. This is actually just another decorator which works by copying the metadata from your wrapped function over to the wrapper function.

Let's modify our decorator to fix this problem...


In [5]:

import functools


def my_decorator(func):
    """
    Decorator to add information about which function was called
    """
    @functools.wraps(func) ####### NOTICE THIS NEW LINE
    def wrapper(*args, **kwargs):
        print(f"{func.__qualname__}")
        return func(*args, **kwargs)
    return wrapper


class MySillyClass:
    """
    A silly class; perfect for a tutorial
    """

    @my_decorator
    def foo(self):
        """
        Beautfil is better than ugly.
        """
        return "foo"

    @my_decorator
    def bar(self):
        """
        Explicit is better than implicit.
        """
        return "bar"

    @my_decorator
    def baz(self):
        """
        Simple is better than complex.
        """
        return "baz"

    
print(help(MySillyClass))

Help on class MySillyClass in module __main__:

class MySillyClass(builtins.object)
 |  A silly class; perfect for a tutorial
 |  
 |  Methods defined here:
 |  
 |  bar(self)
 |      Explicit is better than implicit.
 |  
 |  baz(self)
 |      Simple is better than complex.
 |  
 |  foo(self)
 |      Beautfil is better than ugly.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)

None



Now that we've managed to fix our simple decorator it's time to look into some more advanced usage.

Consider the scenario that we've built a whole bunch of classes that live in different modules across our codebase. They are all relying on our decorator, but there's one problem. We've moved from using print to using loggers, yet our decorator is still using print to add information!

For example, if we configure the root logger in our Python process:


In [6]:

# Configure the root logger:
import logging
import sys
LOGGER_CONFIG_FORMAT = "[%(asctime)s] [%(name)s] %(levelname)-5s: %(message)s"
LOGGER_CONFIG_DATEFMT = "%Y-%m-%d %H:%M:%S"
LOGGER_CONFIG_STREAM = sys.stdout
logging.basicConfig(
    level=logging.DEBUG,
    format=LOGGER_CONFIG_FORMAT,
    datefmt=LOGGER_CONFIG_DATEFMT,
    stream=LOGGER_CONFIG_STREAM)


In [7]:

# And we create a logger for our silly class:
SILLY_LOGGER = logging.getLogger("SILLY")



Our silly class wants to use the SILLY logger, yet our decorator is hard-coded to use print to output some information.

Let's modify our decorator....


In [8]:
import functools


def my_decorator(logger):
    """
    Decorate a function using a specific logger
    """
    def decorate(func):
        """
        Decorator to add information about which function was called
        """
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            logger.info("%s", func.__qualname__)
            return func(*args, **kwargs)
        return wrapper
    return decorate


class MySillyClass:
    """
    A silly class; perfect for a tutorial
    """

    @my_decorator(SILLY_LOGGER)
    def foo(self):
        """
        Beautfil is better than ugly.
        """
        return "foo"

    @my_decorator(SILLY_LOGGER)
    def bar(self):
        """
        Explicit is better than implicit.
        """
        return "bar"

    @my_decorator(SILLY_LOGGER)
    def baz(self):
        """
        Simple is better than complex.
        """
        return "baz"


In [9]:

# Create a new instance of our silly class and invoke it's methods:
silly = MySillyClass()
print(f"silly.foo = {silly.foo()}\n")
print(f"silly.bar = {silly.bar()}\n")
print(f"silly.baz = {silly.baz()}\n")


[2019-08-09 15:49:53] [SILLY] INFO : MySillyClass.foo
silly.foo = foo

[2019-08-09 15:49:53] [SILLY] INFO : MySillyClass.bar
silly.bar = bar

[2019-08-09 15:49:53] [SILLY] INFO : MySillyClass.baz
silly.baz = baz




Looking good. Now the decorator is using a logger rather than print, but what if we have some areas of our codebase which haven't configured a logger? The decorator should always fallback to a default value, for example:


In [10]:

import logging

def my_decorator(logger=logging.getLogger("DEFAULT")):
    """
    Decorate a function using a specific logger
    """
    def decorate(func):
        """
        Decorator to add information about which function was called
        """
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            logger.info("%s", func.__qualname__)
            return func(*args, **kwargs)
        return wrapper
    return decorate



Let's consider the original implementation of our class, where we did not have to specify a logger to the decorator:


In [11]:

class MySillyClass:
    """
    A silly class; perfect for a tutorial
    """

    @my_decorator
    def foo(self):
        """
        Beautfil is better than ugly.
        """
        return "foo"

    @my_decorator
    def bar(self):
        """
        Explicit is better than implicit.
        """
        return "bar"

    @my_decorator
    def baz(self):
        """
        Simple is better than complex.
        """
        return "baz"


In [12]:

# Lets create an instance and call one of the methods...
silly = MySillyClass()
silly.foo()


<function __main__.my_decorator.<locals>.decorate.<locals>.wrapper>


##### Confused? I'm going to try to explain what's going on:

We've now instructed Python to get a method named "my_decorator", and use that method to decorate our function.

However, we've modified "my_decorator" so it no longer decorates a function, but returns to us a function (named decorate) that is able to decorate our function. In making this change, my_decorator is no longer able to do the decorating itself because it doesn't know about the function being decorated; how can it? It no longer has an arg for the function-to-be-wrapped.


##### Why no Exception?

If you've been paying very close attention you'll notice that the "decorate" function takes an arg named *func* yet we are not passing this arg! So why is there no TypeError Exception being raised?

The reason is simple: remember that Python will always send self as the first arg to any bound class method, in other words:

silly.foo() == Silly.foo(silly)

In our case, what we are actually calling is decorate(silly), which returns to us a function called "wrapper", hence the funny error seen above.

To further explain this, let's decorate a global function in the same way:



In [13]:
@my_decorator
def some_global_func():
    print("I AM A GLOBAL")

# Try calling the global function...
some_global_func()

TypeError: decorate() missing 1 required positional argument: 'func'


Now you'll see that we get a TypeError telling us that *func* was not passed - because global functions are not bound methods, so no default self arg is passed!

Don't be worried if you're really confused by now because it's fairly tricky to understand and even trickier to explain - so please bear with me, and try re-reading, and even playing around with the code yourself.


##### How can this be fixed???

Using another nifty, (but slightly confusing) Python standard library concept named partial!

Have a look at this new decorator implementation and I'll try to explain how it works:


In [14]:

from functools import partial


def my_decorator(func=None, logger=logging.getLogger("DEFAULT")):
    # Here we are saying if the function is not passed, return a partial
    # function of this decorator, which can then be used to decorate!
    if not func:
        return partial(my_decorator, logger=logger)
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        logger.debug(f"{func.__qualname__}")
        return func(*args, **kwargs)
    return wrapper


@my_decorator
def some_global_func():
    print("I AM A GLOBAL")

# Try calling the global function...
some_global_func()


[2019-08-09 15:50:00] [DEFAULT] DEBUG: some_global_func
I AM A GLOBAL



##### How does this work?

The best way to show how this works is by writing out some simple Python:


In [15]:

# Start with an un-decorated simple function:
def another_global_func():
    print("ANOTHER GLOBAL FUNC")

# Call our decorator function to get a partial decorator.
# This will return a partial function because we did not specify
# a value for the func argument when calling my_decorator():
partial_decorator = my_decorator()

# Now we can "manually" decorate our un-decorated function by 
# simply calling the partial function and passing in the 
# "function-to-decorate":
decorated_function = partial_decorator(another_global_func)

# Finally, lets call the decorated version:
decorated_function()


[2019-08-09 15:50:02] [DEFAULT] DEBUG: another_global_func
ANOTHER GLOBAL FUNC




## Summary:

We've covered both simple and advanced decorators, hopefully you've now got a good understanding of what is going on as opposed to considering these concepts "magic"!

I'll finish off by giving you some boiler-plate code for both types of decorator covered by this tutorial. Feel free to use these templates to roll your own decorators.


In [16]:

def simple_decorator(func):
    """
    Boiler plate code for a simple decorator
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # ADD CODE BEFORE CALLING DECORATED FUNCTION
        result = func(*args, **kwargs)
        # ADD CODE AFTER CALLING DECORATED FUNCTION
        return result
    return wrapper


def advanced_decorator(func=None, logger=logging.getLogger("DEFAULT")):
    """
    Boiler plate code for a decorator which takes optional arguments
    """
    if not func:
        return partial(my_decorator, logger=logger)
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # ADD CODE BEFORE CALLING DECORATED FUNCTION
        result = func(*args, **kwargs)
        # ADD CODE AFTER CALLING DECORATED FUNCTION
        return result
    return wrapper
